In [1]:
from functools import reduce
from json import loads

import pandas as pd
import requests
from fake_useragent import UserAgent

In [20]:
ua = UserAgent()
url = "https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=12-15-2024"
headers = {"User-Agent": ua.random}

response = requests.get(url, headers=headers)
startidx = response.text.find("(")
endidx = response.text.find(")")
data = loads(response.text[startidx + 1 : endidx])
series_list = loads(data["json"])

In [21]:
df_list = []
for series in series_list:
    df_dict = {}
    region = series["label"]
    df_dict["Timestamp"] = [
        pd.to_datetime(x[0], unit="ms") for x in series["data"]
    ]
    df_dict[region] = [int(x[1]) for x in series["data"]]
    df_list.append(pd.DataFrame(df_dict))

df = reduce(
    lambda x, y: pd.merge(x, y, on="Timestamp", how="outer"), df_list
)
df = df.sort_values("Timestamp").reset_index(drop=True)

In [22]:
df

,Timestamp,Central America,Africa,Middle East,Oceania,South America,Russia,Asia,Europe,North America
0,2024-12-12 02:00:00,159,66,467,535,3118,507,4780,2276,11929
1,2024-12-12 02:10:00,158,65,454,535,3059,507,4780,2223,11929
2,2024-12-12 02:20:00,158,65,454,551,3065,514,5070,2251,12104
3,2024-12-12 02:30:00,158,65,454,572,3059,514,5399,2223,12104
4,2024-12-12 02:40:00,159,62,430,575,2923,504,5655,2098,12208
...,...,...,...,...,...,...,...,...,...,...
282,2024-12-14 01:00:00,167,92,707,713,3450,774,4956,4654,11998
283,2024-12-14 01:10:00,167,87,689,740,3452,756,5313,4424,12151
284,2024-12-14 01:20:00,167,87,670,745,3452,742,5739,4195,12279
285,2024-12-14 01:30:00,170,84,644,756,3434,719,6131,4005,12307


In [23]:
df.loc[df["Timestamp"] == pd.Timestamp("2024-12-13 04:40:00")]

,Timestamp,Central America,Africa,Middle East,Oceania,South America,Russia,Asia,Europe,North America
160,2024-12-13 04:40:00,159,71,361,850,1803,679,10035,1879,11966
